# Use Wikipedia as alternative data source

In [128]:
import numpy as np
import pandas as pd
import wikipedia
import re
import timeit
from bs4 import BeautifulSoup

## Load Formated Data

In [3]:
country_df = pd.read_pickle("../../data/Tables/country_df.pkl")
comes_from_df = pd.read_pickle("../../data/Tables/comes_from_df.pkl")
genre_df = pd.read_pickle("../../data/Tables/genre_df.pkl")
is_of_type_df = pd.read_pickle("../../data/Tables/is_of_type_df.pkl")
language_df = pd.read_pickle("../../data/Tables/language_df.pkl")
spoken_languages_df = pd.read_pickle("../../data/Tables/spoken_languages_df.pkl")
character_df = pd.read_pickle("../../data/Tables/character_df.pkl")
actor_df = pd.read_pickle("../../data/Tables/actor_df.pkl")
movie_df = pd.read_pickle("../../data/Tables/movie_df.pkl")
belongs_to_df = pd.read_pickle("../../data/Tables/belongs_to_df.pkl")
play_df = pd.read_pickle("../../data/Tables/play_df.pkl")
appears_in_df = pd.read_pickle("../../data/Tables/appears_in_df.pkl")

## Test

In [13]:
movie_df

,name,release_date,revenue,runtime,freebase_id,plot
movie_id,,,,,,
975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,/m/03vyhn,"Set in the second half of the 22nd century, th..."
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,/m/08yl5d,NaN
28463795,Brun bitter,1988,NaN,83.0,/m/0crgdbh,NaN
9363483,White Of The Eye,1987,NaN,110.0,/m/0285_cd,A series of murders of rich young women throug...
261236,A Woman in Flames,1983,NaN,106.0,/m/01mrr1,"Eva, an upper class housewife, becomes frustra..."
...,...,...,...,...,...,...
35228177,Mermaids: The Body Found,2011-03-19,NaN,120.0,/m/0j7hxnt,Two former National Oceanic Atmospheric Admini...
34980460,Knuckle,2011-01-21,NaN,96.0,/m/0g4pl34,{{No plot}} This film follows 12 years in the ...
9971909,Another Nice Mess,1972-09-22,NaN,66.0,/m/02pygw1,NaN


In [224]:
p = wikipedia.page(pageid=261236)

In [225]:
soup = BeautifulSoup(p.html(),"html.parser")

In [230]:
table = soup.find_all("table",class_="infobox")[0]

In [237]:
table

<table class="infobox vevent"><tbody><tr><th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">A Woman in Flames</th></tr><tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:A_Woman_in_Flames_dvd_cover.jpg"><img alt="A Woman in Flames dvd cover.jpg" class="thumbborder" data-file-height="360" data-file-width="256" decoding="async" height="309" src="//upload.wikimedia.org/wikipedia/en/thumb/5/55/A_Woman_in_Flames_dvd_cover.jpg/220px-A_Woman_in_Flames_dvd_cover.jpg" srcset="//upload.wikimedia.org/wikipedia/en/5/55/A_Woman_in_Flames_dvd_cover.jpg 1.5x" width="220"/></a><div class="infobox-caption">German language dvd cover art</div></td></tr><tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Directed by</th><td class="infobox-data"><a href="/wiki/Robert_van_Ackeren" title="Robert van Ackeren">Robert van Ackeren</a></td></tr><tr><th class="infobox-label" scope="row" style="white-space: no